In [1]:
%pylab inline
import json
import numpy as np
import pandas as pd
import resource
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from sklearn.metrics.pairwise import pairwise_distances
from multiprocessing.dummy import Pool as ThreadPool

Populating the interactive namespace from numpy and matplotlib


In [5]:
tf = TfidfVectorizer(lowercase=False, preprocessor=None).fit(map(lambda x: x.strip().split('\t')[-1], 
                                                                 open('facts.txt').readlines()))

In [6]:
users, row_text = [], []
with open('facts_test.tsv') as f_in:
    for line in tqdm(f_in):
        user, t = line.strip().split('\t')
        users.append(user)
        row_text.append(t)

In [7]:
tf_test = tf.transform(row_text)

In [8]:
tf_test.shape

(98673, 336949)

In [12]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(tf_test, range(1, tf_test.shape[0] + 1))
pool = ThreadPool(16)

In [67]:
def get_predict(line):
    res = []
    user_id, tokens = line.strip().split('\t')
    cur_vect = tf.transform([tokens])
    tmp = knn.kneighbors(X=tf.transform([tokens]), n_neighbors=15, return_distance=True)
    for i in range(len(tmp[0][0])):
        if 0 < tmp[0][0][i] < 0.9999:
            if user_id < users[tmp[1][0][i]]:
                res.append((user_id, users[tmp[1][0][i]], tmp[0][0][i]))
            else:
                res.append((users[tmp[1][0][i]], user_id, tmp[0][0][i]))
    return res

In [ ]:
%%time
lines = open('facts_test.tsv').readlines()
results = pool.map(get_predict, lines)

In [79]:
%%time
q = sorted(list(set([y for x in results for y in x])), key=lambda x: x[-1])

CPU times: user 2.36 s, sys: 32.1 ms, total: 2.39 s
Wall time: 2.36 s


In [83]:
with open('submission_all.csv','w') as f_out:
    for x in tqdm(q[:300000]):
        f_out.write("%s,%s\n" % (x[0], x[1]))